## 使用 pdf2markdown4llm 將 pdf 轉 markdown 格式

In [1]:
from pdf2markdown4llm import PDF2Markdown4LLM

import pymupdf4llm

### 定義進度回呼函式
建立一個 progress_callback 函式，可以在轉檔時即時顯示當前階段、處理頁碼、進度百分比與提示訊息。

In [2]:
def progress_callback(progress): 
    """Callback function to handle progress"""
    # 轉檔過程中，每個階段都會呼叫這個函式，顯示目前進度與訊息
    print(f"Phase: {progress.phase.value}, Page {progress.current_page}/{progress.total_pages}, Progress: {progress.percentage:.1f}%, Message: {progress.message}")

### 初始化轉換器 (Converter)
設定轉檔時的一些選項。例如是否要移除頁首（headers）、略過空表格、表格標題的格式等，並連結上方定義的進度回呼。

In [3]:
# 初始化轉換器，指定轉檔選項與進度回呼
converter = PDF2Markdown4LLM(
    remove_headers=False,        # 不移除文件頁首
    skip_empty_tables=True,      # 略過內容為空的表格
    table_header="### Table",    # 表格前新增自訂 Markdown 標題
    progress_callback=progress_callback  # 設定進度回呼函式
)

### 執行轉檔
使用 converter.convert("input.pdf") 進行 PDF 到 Markdown 的轉換，轉換進度會自動調用前述回呼函式。

In [4]:
# 指定你的 PDF 檔案路徑
pdf_path = "永續資料_與中信有關\永續管理政策.pdf"

In [6]:
# 處理第1頁（Python中的頁碼從0開始）
md_text = pymupdf4llm.to_markdown(pdf_path, pages=[0])
md_text

Processing 永續資料_與中信有關\永續管理政策.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]


'```\n中國信託金融控股股份有限公司\n    永續管理政策\n        版本<2.0>\n    中國信託金融控股(股)公司\n           著作權聲明\n本文件所載之各項著作內容皆屬中國信託金融控股(股)公司所有，皆受到中華\n 民國著作權法的保護，未經書面同意不得以任何形式修改、複製及轉載。\n\n```\n管制文件, 禁止翻印\n\n```\n            著作權聲明\n 本文件所載之各項著作內容皆屬中國信託金融控股(股)公司所有，皆受到中華\n  民國著作權法的保護，未經書面同意不得以任何形式修改、複製及轉載。\n\n```\n\n-----\n\n'

分開輸出（分chunk）

In [ ]:
data = pymupdf4llm.to_markdown(pdf_path, page_chunks=True)
# data[0]["text"]  # 取得第一頁的 Markdown 內容
# for i, page in enumerate(data):
#     with open(f"PDF_To_Markdown/page_{i+1}.md", "w", encoding="utf-8") as f:
#         f.write(page["text"])


Processing 永續資料_與中信有關\永續管理政策.pdf...
[                                        ] (0/5=======[========                                ] (1/5=======[================                        ] (2/5=======[========================                ] (3/5=======[================================        ] (4/5=======[========================================] (5/5]


查看單一頁內容

In [15]:
# 第三頁內容

third_page_md = data[2]["text"]  # data[2] 是第三頁
print(third_page_md)

```
文件等級：公開

```
```
   中國信託金融控股股份有限公司
       永續管理政策
第一條 制定依據及目的
    中國信託金融控股(股)公司（以下稱「中信金控」或「本公司」）承諾
    持續呼應聯合國永續發展目標(Sustainable Development Goals)，在兼顧
    本公司業務成長、利害關係人（Stakeholder）權益以及環境、社會的
    永續發展之下，訂定中信金控永續管理政策（以下稱「本政策」），作為
    落實永續經營之遵循方針，並朝向2050淨零之目標邁進。

```
`本政策酌參聯合國「永續發展目標(Sustainable` Development Goals)」、「巴黎

`協定(Paris Agreement)` `」、「全球盟約` (Global Compact) 」、「責任銀行原

`則` (Principles for Responsible Banking) 」、「責任投資原則(Principles for

Responsible Investment) 」、「 `永` `續` `保` `險` `原` `則 (Principles for Sustainable`

Insurance)」、「科學基礎減量目標(Science-based targets, SBT)」等國際準
```
    則、協定與倡議，以及主管機關相關政策及法令。未盡規範事宜，另參
    照本公司其他政策或辦法辦理。
第二條 適用範圍
    本政策適用於本公司及各子公司。
第三條 管理架構
    一、本政策權責單位為本公司企業永續部。
    二、本公司建置永續管理組織如下圖，以落實本政策的推行。「企業永
      續委員會」為董事會功能性委員會，另於經營管理階層下設
      「ESG 執行小組」，依「公司治理、環境永續、員工照顧、永續金
      融、社會參與及客戶關懷」六大面向，偕同各子公司經營管理階層，
      訂定企業永續發展各範疇年度目標並定期進行追蹤控管。
    三、企業永續部得彙整相關制定情形及實施成效，呈報董事會。

```
3


-----




### LlamaIndex 支援
專為 LLM/RAG 框架資料流程設計，讓 PDF 內容直接變成可用於資訊檢索的 document list 物件，含頁碼與摘要等 metadata

In [5]:
def clean_markdown(raw_md):
    # 去掉多個 ```
    import re
    # 清除所有獨立行內 ``` 或 ```
    cleaned = re.sub(r'^\s*`{3,}\s*$', '', raw_md, flags=re.MULTILINE)
    # 合併被斷行的句子
    cleaned = re.sub(r'([^\n])\n([^\n])', r'\1\2', cleaned)
    # 可根據 PDF 條例格式繼續調整
    return cleaned.strip()

In [6]:
llama_reader = pymupdf4llm.LlamaMarkdownReader()
llama_docs = llama_reader.load_data(pdf_path)

Successfully imported LlamaIndex
Processing 永續資料_與中信有關\永續管理政策.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing 永續資料_與中信有關\永續管理政策.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing 永續資料_與中信有關\永續管理政策.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing 永續資料_與中信有關\永續管理政策.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing 永續資料_與中信有關\永續管理政策.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]


In [9]:
llama_docs[3]

Document(id_='f98b6db8-9e6d-4080-8478-c2387f44244e', embedding=None, metadata={'format': 'PDF 1.7', 'title': '中信金控永續管理政策', 'author': 'z09562', 'subject': '', 'keywords': '', 'creator': '適用於 Microsoft 365 的 Microsoft® Word', 'producer': '適用於 Microsoft 365 的 Microsoft® Word', 'creationDate': "D:20240801135325+08'00'", 'modDate': "D:20240801135325+08'00'", 'trapped': '', 'encryption': None, 'page': 4, 'total_pages': 5, 'file_path': '永續資料_與中信有關\\永續管理政策.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='```\n文件等級：公開\n\n```\n```\n第四條 永續策略與發展目標\n    本公司以「永續成長」、「責任營運」及「共榮社會」為永續發展三大策略面\n    向，並承諾以此作為管理方針，促進符合集團價值的永續文化，為本公司\n    利害關係人創造長期之共享價值：\n    一、永續成長\n      結合永續發展及數位科技趨勢，本公司督導子公司持續精進創新金\n      融商品、服務與作業品質，達成永續成長。承諾：\n\n```\n(一)、致力開發與聯合國永續發展目標連結之金融商品與服務，包\n```\n        含但不限於包容性金融(Inclusive Finance)及低碳經濟領域，\n         期透過資金運用， 打造正向影響(positive impact)金融圈。\n\n```\n(二)、運用數位技術，創建人人可以取得、使用的金融系統，提供客\n```\n        戶更便捷、創新及安全的金融服務。\n    二、責任營運\n      良好的職場環

In [25]:
output_md = clean_markdown(llama_docs[3].text)
print(output_md)

文件等級：公開


第四條 永續策略與發展目標    本公司以「永續成長」、「責任營運」及「共榮社會」為永續發展三大策略面    向，並承諾以此作為管理方針，促進符合集團價值的永續文化，為本公司    利害關係人創造長期之共享價值：    一、永續成長      結合永續發展及數位科技趨勢，本公司督導子公司持續精進創新金      融商品、服務與作業品質，達成永續成長。承諾：

(一)、致力開發與聯合國永續發展目標連結之金融商品與服務，包

        含但不限於包容性金融(Inclusive Finance)及低碳經濟領域，         期透過資金運用， 打造正向影響(positive impact)金融圈。

(二)、運用數位技術，創建人人可以取得、使用的金融系統，提供客

        戶更便捷、創新及安全的金融服務。    二、責任營運      良好的職場環境、企業文化，以及健全的公司治理制度是本公司永續      經營的根本。本公司督導子公司在自身營運過程中，致力於降低所產      生的直接、間接環境衝擊，以落實整體責任營運。承諾：

(一)、視人才為公司永續發展之關鍵資本。透過平等的工作環境

4


-----


## 使用 pdfplumber 工具將 pdf 轉文字內容

In [19]:
import pdfplumber

with pdfplumber.open(pdf_path) as pdf:
    first_page = pdf.pages[0]
    text = first_page.extract_text()
    print(text)


中國信託金融控股股份有限公司
永續管理政策
版本<2.0>
中國信託金融控股(股)公司
著作權聲明
本文件所載之各項著作內容皆屬中國信託金融控股(股)公司所有，皆受到中華
民國著作權法的保護，未經書面同意不得以任何形式修改、複製及轉載。
管制文件, 禁止翻印


In [ ]:
# 將指定 PDF 檔轉為 Markdown 格式字串
markdown_content = converter.convert(pdf_path)

Phase: analysis, Page 0/5, Progress: 0.0%, Message: Starting PDF analysis
Phase: analysis, Page 1/5, Progress: 14.0%, Message: Analyzing
Phase: analysis, Page 2/5, Progress: 28.0%, Message: Analyzing
Phase: analysis, Page 3/5, Progress: 42.0%, Message: Analyzing
Phase: analysis, Page 4/5, Progress: 56.0%, Message: Analyzing
Phase: analysis, Page 5/5, Progress: 70.0%, Message: Analyzing
Phase: conversion, Page 0/5, Progress: 70.0%, Message: Analysis complete, beginning content extraction
Phase: conversion, Page 1/5, Progress: 76.0%, Message: Converting content to Markdown
Phase: conversion, Page 2/5, Progress: 82.0%, Message: Converting content to Markdown
Phase: conversion, Page 3/5, Progress: 88.0%, Message: Converting content to Markdown
Phase: conversion, Page 4/5, Progress: 94.0%, Message: Converting content to Markdown
Phase: conversion, Page 5/5, Progress: 100.0%, Message: Converting content to Markdown
Phase: conversion, Page 5/5, Progress: 100.0%, Message: Conversion complete


In [20]:
# 只處理第 1 頁（Python 的頁碼從 0 開始計算）
md_text = PDF2Markdown4LLM.to_markdown(pdf_path, pages=[0])

AttributeError: type object 'PDF2Markdown4LLM' has no attribute 'to_markdown'

### 儲存結果
將轉換後的 Markdown 內容寫入 output.md，以 UTF-8 編碼儲存。

In [ ]:
# 將轉換後的 Markdown 內容寫入 .md 檔
with open("output.md", "w", encoding="utf-8") as md_file:
    md_file.write(markdown_content)